In [48]:
import numpy as np
import re
import json
from collections import deque

In [56]:
def node_embed(node_param, max_filter, num_filter_attr, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, num_attr):

  # operations
  op_vec = np.array([0 for _ in range(len(op_stats))])
  op_vec[op_stats[node_param['op_name']]] = 1

  # attributes
  attr_vec = np.array([0.0 for _ in range(num_attr)])
  attr_vec[0] = node_param['est_card']
  attr_vec[1] = node_param['est_width']
  if 'table' in node_param:
    table_name = table_id_map[str(node_param['table'])]
    table_stat = table_stats[table_name]
    attr_vec[2] = table_stat['reltuples']
    attr_vec[3] = table_stat['relpages']


  # predicates
  # col filter_op literal
  filter_vec = np.array([0.0 for _ in range(max_filter * num_filter_attr)])
  if 'filter_columns' in node_param:
    filter_idx = 0
    num_data_type = len(type_id_map)
    # 5 col statistics are used
    num_col_fileds = num_data_type + 5
    num_filter_op = len(filter_op_id_map)

    filters = [node_param['filter_columns']]
    if filters[0]['operator'] == "AND":
      filters = node_param['filter_columns']['children']

    for fil in filters:
      print(fil)
      # vector includes:
      # column related:
      #   col data type, num_unique, nan_ratio, attr correlation with row number, avg width, table_size
      # filter_op
      # literal_related: unique values of literal
      fil_start = filter_idx * num_filter_attr
      col_stat = col_stats[col_id_map[str(fil['column'])]]
      col_data_type = col_stat["datatype"]
      filter_vec[fil_start + type_id_map[col_data_type]] = 1
      filter_vec[fil_start + num_data_type] = col_stat["num_unique"]
      filter_vec[fil_start + num_data_type + 1] = col_stat["nan_ratio"]
      filter_vec[fil_start + num_data_type + 2] = col_stat["corr_with_row"]
      filter_vec[fil_start + num_data_type + 3] = col_stat["avg_width"]
      filter_vec[fil_start + num_data_type + 4] = col_stat["table_size"]

      filter_op_idx = filter_op_id_map[fil['operator']]
      filter_vec[fil_start + num_col_fileds + filter_op_idx] = 1

      literal = fil['literal']
      filter_vec[fil_start + num_col_fileds + num_filter_op] = 1
      if isinstance(literal, str):
        l_col = literal.strip()
        if re.match(r'^\w+\.\w+$', l_col):
          filter_vec[fil_start + num_col_fileds + num_filter_op] = col_stats[l_col]["num_unique"]

      filter_idx += 1

  # outputs
  # output aggregation type inclue COUNT, AVG, SUM, and NONE
  num_col = len(col_id_map)
  output_vec = np.array([0 for _ in range(num_col * 3 + 1)])
  output_avg_start = 1
  output_sum_start = num_col + 1
  output_none_start = 2*num_col + 1
  if 'output_columns' in node_param:
    for output in node_param['output_columns']:
      agg_op = output['aggregation']

      if agg_op == 'COUNT':
        output_vec[0] = 1
      elif agg_op == 'AVG':
        for output_col in output['columns']:
          output_vec[output_avg_start + output_col] = 1
      elif agg_op == "SUM":
        for output_col in output['columns']:
          output_vec[output_sum_start + output_col] = 1
      else:
        for output_col in output['columns']:
          output_vec[output_none_start + output_col] = 1

  return op_vec, attr_vec, filter_vec, output_vec

In [4]:

with open("../../../datasets/stats/stat_complete.json", "r") as f:
  stats = json.load(f)

In [5]:
len(stats)
node_param, max_filter, num_filter_attr, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, num_attr

7

In [16]:
col_id_map, col_stats, table_id_map, table_stats, type_id_map, op_stats, filter_op_id_map = stats

In [8]:
parsed_5k = "../../../datasets/plans/parsed/workload_5k_s1_c8220.json"
with open(parsed_5k, "r") as f:
    parsed_5k_plan = json.load(f)

In [14]:
node_param = parsed_5k_plan['parsed_plans'][0]['children'][0]['plan_parameters']

In [49]:
for plan in parsed_5k_plan['parsed_plans']:
  level = 0

  queue = deque([plan])

  while queue:
    level_count = len(queue)
    while level_count > 0:
      level_count -= 1
      node = queue.popleft()
      param = node['plan_parameters']

      if 'filter_columns' in param:
        # print(param['filter_columns'])
        op = param['filter_columns']['operator']
        if op == "AND":
          node_param = param
          break

      for children in node['children']:
        queue.append(children)


In [50]:
node_param

{'table': 6,
 'op_name': 'Seq Scan',
 'est_startup_cost': 0.0,
 'est_cost': 48595.0,
 'est_card': 1192098.0,
 'est_width': 8.0,
 'act_startup_cost': 0.012,
 'act_time': 399.803,
 'act_card': 1198171.0,
 'filter_columns': {'column': None,
  'operator': 'AND',
  'literal': None,
  'literal_feature': None,
  'children': [{'column': 42,
    'operator': '!=',
    'literal': 'Clerk#000000918',
    'literal_feature': 0,
    'children': []},
   {'column': 41,
    'operator': '!=',
    'literal': '5-LOW',
    'literal_feature': 0,
    'children': []}]},
 'output_columns': [{'aggregation': 'None', 'columns': [36]},
  {'aggregation': 'None', 'columns': [37]},
  {'aggregation': 'None', 'columns': [38]},
  {'aggregation': 'None', 'columns': [39]},
  {'aggregation': 'None', 'columns': [40]},
  {'aggregation': 'None', 'columns': [41]},
  {'aggregation': 'None', 'columns': [42]},
  {'aggregation': 'None', 'columns': [43]},
  {'aggregation': 'None', 'columns': [44]}],
 'act_children_card': 1,
 'est_chi

In [57]:
op_vec, attr_vec, filter_vec, output_vec = node_embed(node_param, 5, 14, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, 4)

{'column': 42, 'operator': '!=', 'literal': 'Clerk#000000918', 'literal_feature': 0, 'children': []}
{'column': 41, 'operator': '!=', 'literal': '5-LOW', 'literal_feature': 0, 'children': []}


In [58]:
print(filter_vec[:14])
print(filter_vec[14:28])

[0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 1.00000e+03 0.00000e+00
 8.56727e-03 1.60000e+01 1.50000e+06 0.00000e+00 0.00000e+00 0.00000e+00
 1.00000e+00 1.00000e+00]
[0.00000e+00 1.00000e+00 0.00000e+00 0.00000e+00 5.00000e+00 0.00000e+00
 1.99784e-01 1.60000e+01 1.50000e+06 0.00000e+00 0.00000e+00 0.00000e+00
 1.00000e+00 1.00000e+00]


In [60]:
print(len(output_vec))
print(output_vec[0])
print(output_vec[1:62])
print(output_vec[62:123])
print(output_vec[123:])

184
0
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1
 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [99]:
def plan_embed(root_node, max_filter, num_filter_attr, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, num_attr):

  node_queue = deque([root_node])

  plan_op, plan_attr, plan_filter, plan_output, plan_mapping = [], [], [], [], []

  while node_queue:
    level_count = len(node_queue)

    plan_op.append([])
    plan_attr.append([])
    plan_filter.append([])
    plan_output.append([])
    plan_mapping.append([])

    children_sum = 0

    while level_count > 0:
      level_count -= 1
      node = node_queue.popleft()
      node_param = node.plan_parameters
      op_vec, attr_vec, filter_vec, output_vec = node_embed(node_param, max_filter, num_filter_attr, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, num_attr)

      plan_op[-1].append(op_vec)
      plan_attr[-1].append(attr_vec)
      plan_filter[-1].append(filter_vec)
      plan_output[-1].append(output_vec)

      if len(node.children) == 2:
        plan_mapping[-1].append([child.idx + children_sum for child in node.children])
      elif len(node.children) == 1:
        plan_mapping[-1].append([node.children[0].idx + children_sum, 0])
      else:
        plan_mapping[-1].append([0,0])

      children_sum += len(node.children)

      for child in node.children:
        node_queue.append(child)

  return plan_op, plan_attr, plan_filter, plan_output, plan_mapping

In [94]:
from typing import Optional, List, Dict

class SQLNode():
    def __init__(self, plain_content:List, plan_parameters:Dict,  plan_runtime: float) -> None:
        self.plain_content = plain_content
        self.plan_parameters = plan_parameters
        self.children : Optional[List['SQLNode']] = []
        self.plan_runtime = plan_runtime
        self.idx = 1
        # self.join_conds = join_conds


class SQLTree():
    def __init__(self, parsed_sql:Dict) -> None:
        plain_content = parsed_sql["plain_content"]
        plan_parameters = parsed_sql["plan_parameters"]
        children = parsed_sql["children"]
        plan_runtime = parsed_sql["plan_runtime"]
        # join_conds = parsed_sql["join_conds"]
        self.root : Optional[SQLNode] = SQLNode(plain_content, plan_parameters, plan_runtime)
        self._insert_children_sql(self.root, children)
        
    def _insert_children_sql(self, node: SQLNode, children):
        for parsed_sql in children:
            plain_content = parsed_sql["plain_content"]
            plan_parameters = parsed_sql["plan_parameters"]
            children = parsed_sql["children"]
            plan_runtime = parsed_sql["plan_runtime"]
            # join_conds = parsed_sql["join_conds"]
            new_node = SQLNode(plain_content, plan_parameters, plan_runtime)
            new_node.idx = len(node.children) + 1
            node.children.append(new_node)
            self._insert_children_sql(new_node, children)
        
        return

class workload_dataloader():
    def __init__(self, filename) -> None:
        with open(filename, "r") as f:
            data = json.load(f)
        parsed_sqls = data["parsed_plans"]
        self.sql_forest = []
        for parsed_sql in parsed_sqls:
            tree = SQLTree(parsed_sql)
            self.sql_forest.append(tree)
    
    def get_data(self):
        return self.sql_forest

In [95]:
filename = "../../../datasets/plans/parsed/workload_5k_s1_c8220.json"
workload = workload_dataloader(filename)
tree = workload.get_data()[1293]

In [96]:
root_node = tree.root

In [100]:
plan_op, plan_attr, plan_filter, plan_output, plan_mapping = plan_embed(root_node, 5, 14, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, 4)

{'column': 37, 'operator': '=', 'literal': 'customer.c_custkey  ', 'literal_feature': 0, 'children': []}
{'column': 40, 'operator': '!=', 'literal': '1992-07-27', 'literal_feature': 0, 'children': []}
{'column': 42, 'operator': '=', 'literal': 'Clerk#000000682', 'literal_feature': 0, 'children': []}
{'column': 28, 'operator': '>=', 'literal': 120925.0, 'literal_feature': 0, 'children': []}
{'column': 31, 'operator': '<=', 'literal': 0.0, 'literal_feature': 0, 'children': []}


In [101]:
print(len(plan_mapping))
for maps in plan_mapping:
  print(maps)

5
[[1, 0]]
[[1, 2]]
[[1, 0], [2, 0]]
[[0, 0], [1, 0]]
[[0, 0]]


In [91]:
plan_output[2][0][123:]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [116]:
for op in plan_op:
  print(op)

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0])]
[array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0])]
[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])]
[array([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
[array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]), array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0])]
[array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]


In [104]:
tree = workload.get_data()[1294]
root_node = tree.root
plan_op, plan_attr, plan_filter, plan_output, plan_mapping = plan_embed(root_node, 5, 14, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, 4)
print(len(plan_mapping))
for maps in plan_mapping:
  print(maps)

{'column': 37, 'operator': '=', 'literal': 'customer.c_custkey  ', 'literal_feature': 0, 'children': []}
{'column': 43, 'operator': '=', 'literal': 0.0, 'literal_feature': 0, 'children': []}
6
[[1, 0]]
[[1, 0]]
[[1, 0]]
[[1, 2]]
[[0, 0], [1, 0]]
[[0, 0]]


In [112]:
def batch_embed(plan_trees, max_filter, num_filter_attr, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, num_attr):

  target_runtime = []
  batch_op, batch_attr, batch_filter, batch_output, batch_mapping = [], [], [], [], []

  for plan_tree in plan_trees:
    plan_root = plan_tree.root
    target_runtime.append(plan_root.plan_runtime)

    plan_op, plan_attr, plan_filter, plan_output, plan_mapping = plan_embed(plan_root, max_filter, num_filter_attr, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, num_attr)

    batch_op = _batch_merge_vec(batch_op, plan_op)
    batch_attr = _batch_merge_vec(batch_attr, plan_attr)
    batch_filter = _batch_merge_vec(batch_filter, plan_filter)
    batch_output = _batch_merge_vec(batch_output, plan_output)

    batch_mapping = _batch_merge_mapping(batch_mapping, plan_mapping)


  max_num_node_per_level = 0
  for op in batch_op:
    max_num_node_per_level = max(max_num_node_per_level, len(op))

  print(max_num_node_per_level)
  
  batch_op_pad = pad_to_max_len(batch_op, max_num_node_per_level)
  batch_attr_pad = pad_to_max_len(batch_attr, max_num_node_per_level)
  batch_filter_pad = pad_to_max_len(batch_filter, max_num_node_per_level)
  batch_output_pad = pad_to_max_len(batch_output, max_num_node_per_level)
  batch_mapping_pad = pad_to_max_len(batch_mapping, max_num_node_per_level)

  return target_runtime, batch_op_pad, batch_attr_pad, batch_filter_pad, batch_output_pad, batch_mapping_pad, batch_op, batch_attr, batch_filter, batch_output, batch_mapping

In [103]:

def _batch_merge_vec(batch_vec, plan_vec):
  # if plan has larger number of levels, create a new level list and append
  for level, level_vec in enumerate(plan_vec):
    if level >= len(batch_vec):
      batch_vec.append([])
    batch_vec[level] += level_vec

  return batch_vec

def _batch_merge_mapping(batch_map, plan_map):
  for level, level_map in enumerate(plan_map):
    if level >= len(batch_map):
      batch_map.append([])
    
    # to find if there are preceding parent nodes on the same level
    # if found, should increments its children node mapping (on next level)
    if level < len(batch_map) - 1:
      batch_level_base = len(batch_map[level + 1])
      for i in range(len(level_map)):
        if level_map[i][0] > 0:
          level_map[i][0] += batch_level_base
        if level_map[i][1] > 0:
          level_map[i][1] += batch_level_base
      
    batch_map[level] += level_map

  return batch_map

def pad_to_max_len(batch_vec, max_num):
  return np.array([np.pad(vec, ((0, max_num - len(vec)), (0, 0)), 'constant') for vec in batch_vec])

In [105]:
trees = workload.get_data()[1293:1295]

In [113]:
# plan_op, plan_attr, plan_filter, plan_output, plan_mapping = plan_embed(root_node, 5, 14, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, 4)
target_runtime, batch_op_pad, batch_attr_pad, batch_filter_pad, batch_output_pad, batch_mapping_pad, batch_op, batch_attr, batch_filter, batch_output, batch_mapping = batch_embed(trees, 5, 14, op_stats, col_id_map, col_stats, table_id_map, table_stats, type_id_map, filter_op_id_map, 4)

{'column': 37, 'operator': '=', 'literal': 'customer.c_custkey  ', 'literal_feature': 0, 'children': []}
{'column': 40, 'operator': '!=', 'literal': '1992-07-27', 'literal_feature': 0, 'children': []}
{'column': 42, 'operator': '=', 'literal': 'Clerk#000000682', 'literal_feature': 0, 'children': []}
{'column': 28, 'operator': '>=', 'literal': 120925.0, 'literal_feature': 0, 'children': []}
{'column': 31, 'operator': '<=', 'literal': 0.0, 'literal_feature': 0, 'children': []}
{'column': 37, 'operator': '=', 'literal': 'customer.c_custkey  ', 'literal_feature': 0, 'children': []}
{'column': 43, 'operator': '=', 'literal': 0.0, 'literal_feature': 0, 'children': []}
3


In [120]:
print(len(batch_attr_pad), len(batch_attr))

for i in range(len(batch_attr_pad)):
  level_attr_pad = batch_attr_pad[i]
  level_attr = batch_attr[i]
  print("With pad", len(level_attr_pad), level_attr_pad)
  print("Without pad", len(level_attr), level_attr)


6 6
With pad 3 [[ 1.  8.  0.  0.]
 [ 1. 72.  0.  0.]
 [ 0.  0.  0.  0.]]
Without pad 2 [array([1., 8., 0., 0.]), array([ 1., 72.,  0.,  0.])]
With pad 3 [[11.  0.  0.  0.]
 [ 2. 72.  0.  0.]
 [ 0.  0.  0.  0.]]
Without pad 2 [array([11.,  0.,  0.,  0.]), array([ 2., 72.,  0.,  0.])]
With pad 3 [[1.448e+03 4.000e+00 0.000e+00 0.000e+00]
 [1.118e+03 4.000e+00 0.000e+00 0.000e+00]
 [1.000e+00 7.200e+01 0.000e+00 0.000e+00]]
Without pad 3 [array([1448.,    4.,    0.,    0.]), array([1118.,    4.,    0.,    0.]), array([ 1., 72.,  0.,  0.])]
With pad 3 [[6.0300e+02 4.0000e+00 1.5000e+06 2.6095e+04]
 [1.1180e+03 4.0000e+00 0.0000e+00 0.0000e+00]
 [6.2500e+05 1.6000e+01 0.0000e+00 0.0000e+00]]
Without pad 3 [array([6.0300e+02, 4.0000e+00, 1.5000e+06, 2.6095e+04]), array([1118.,    4.,    0.,    0.]), array([6.25e+05, 1.60e+01, 0.00e+00, 0.00e+00])]
With pad 3 [[4.6600e+02 4.0000e+00 1.5000e+05 3.5850e+03]
 [6.2500e+05 1.6000e+01 1.5000e+06 2.6095e+04]
 [1.5000e+05 4.0000e+00 0.0000e+00 0.0000

In [117]:
print(len(batch_mapping_pad), len(batch_mapping))

for i in range(len(batch_mapping_pad)):
  level_map_pad = batch_mapping_pad[i]
  level_map = batch_mapping[i]
  print("With pad", level_map_pad)
  print("Without pad", level_map)

6 6
With pad [[1 0]
 [2 0]
 [0 0]]
Without pad [[1, 0], [2, 0]]
With pad [[1 2]
 [3 0]
 [0 0]]
Without pad [[1, 2], [3, 0]]
With pad [[1 0]
 [2 0]
 [3 0]]
Without pad [[1, 0], [2, 0], [3, 0]]
With pad [[0 0]
 [1 0]
 [2 3]]
Without pad [[0, 0], [1, 0], [2, 3]]
With pad [[0 0]
 [0 0]
 [1 0]]
Without pad [[0, 0], [0, 0], [1, 0]]
With pad [[0 0]
 [0 0]
 [0 0]]
Without pad [[0, 0]]
